### The simple one directional RNN is implemented here. 
#### The data set used here is as same as the one we used for other networks. Only difference is that instead of considering data in a matix form, the number of rows is considered as number of time steps and the numberof features (columns) is considered as the number of rnn cells.

In [1]:
from math import sqrt
import tensorflow as tf
from tensorflow.contrib import rnn 
from util.data import *

### Parameters ----------------------------------------------------------------------------

In [8]:
learning_rate=0.001
batch_size=256
display_step=100
epochs=1000
Sample_number=10000
Feature_Number=1024

Number_of_RNN_layers=3

#output_keep_prob=0.8   In case you want to apply droup out on cells

DataObj=Data(data_len=Feature_Number,Sample_Number=Sample_number,Test_percentage=40)
Train_x,Train_y,Test_x,Test_y=DataObj.CNN_data()
Number_of_batches=int(len(Train_x)/batch_size)

timesteps=int(sqrt(Feature_Number))
Input_size_for_each_Cell=int(sqrt(Feature_Number))

Number_of_hidden_units_in_eachCell= 200

X=tf.placeholder("float",[None,timesteps,Input_size_for_each_Cell])
Y=tf.placeholder("float",[None,10])

# RE weights, we only need to define the weights of output layer
Weights={'Ow':tf.Variable(tf.random_normal([Number_of_hidden_units_in_eachCell,10],stddev=0.01)),
'Ob':tf.Variable(tf.random_normal([10],stddev=0.01))}


#### Model and structure definition ------------------------------------------------------------------ 

In [9]:
def RNN_Model(X,weights):
    
    # First of all, the input data must be reshaped into the form of timesteps*batchSize*Number_of_rnn_cells
    # the original shape of X is batchSize*timesteps*Number_of_rnn_cells
    
    X=tf.unstack(X,timesteps,1)
    # define a list of rnn cells with tensorflow
    cells=[]
    for i in range(Number_of_RNN_layers):
        cell=rnn.BasicRNNCell(Number_of_hidden_units_in_eachCell)
        # It is also possible to use dropout technique here during cell definition.
        # In the case that we want to apply dropout we can use the follwing instruction before adding a defined cell into the list
        #cell=rnn.DropoutWrapper(cell,output_keep_prob=output_keep_prob)
        cells.append(cell)
        
    rnn_cell=rnn.MultiRNNCell(cells)  
    
    Outputs,States=rnn.static_rnn(rnn_cell,X,dtype=tf.float32)
    out=tf.matmul(Outputs[-1],weights['Ow'])+weights['Ob']
    return tf.nn.softmax(out)

In [10]:
Prediction=RNN_Model(X,Weights)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Prediction,labels=Y))
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
True_Predictions=tf.equal(tf.argmax(Prediction,1),tf.argmax(Y,1))
Accuracy=tf.reduce_mean(tf.cast(True_Predictions,'float'))


#### Graph running --------------------------------------------------------------------------------

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        avg_cost=0
        for j in range(Number_of_batches):
            Trx,Try=next_batch(batch_size, Train_x,Train_y)
            # Reshape Inputs into batch_size*timesteps*Input_size_for_each_Cell form
            Trx=Trx.reshape(batch_size,timesteps,Input_size_for_each_Cell)
            sess.run(train_step,feed_dict={X:Trx,Y:Try})
            avg_cost+=sess.run(cost,feed_dict={X:Trx,Y:Try})/Number_of_batches
        if i%display_step==0:
            
            Train_Accuracy=sess.run(Accuracy,feed_dict={X:Trx,Y:Try})
            Test_x=Test_x.reshape(-1,timesteps,Input_size_for_each_Cell)
            Test_Accuracy=sess.run(Accuracy,feed_dict={X:Test_x,Y:Test_y})
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                   % (i, epochs, avg_cost, Train_Accuracy, Test_Accuracy))
sess.close()